In [7]:
from dgl.contrib.data import load_data
import torch
import pickle

g1 = pickle.load(open("UpdatedGraph_Neighbour10", "rb"))

In [1]:
from vcm.catalog import catalog
import xarray as xr

grid = catalog["grid/c48"].read()


In [2]:
grid.lat[1]

<xarray.DataArray 'lat' (y: 48, x: 48)>
array([[-34.891106, -35.59881 , -36.285767, ..., -36.285767, -35.59881 ,
        -34.891106],
       [-33.414417, -34.10818 , -34.78246 , ..., -34.78246 , -34.10818 ,
        -33.414417],
       [-31.93678 , -32.614727, -33.27444 , ..., -33.27444 , -32.614727,
        -31.93678 ],
       ...,
       [ 31.93678 ,  32.614727,  33.27444 , ...,  33.27444 ,  32.614727,
         31.93678 ],
       [ 33.414417,  34.10818 ,  34.78246 , ...,  34.78246 ,  34.10818 ,
         33.414417],
       [ 34.891106,  35.59881 ,  36.285767, ...,  36.285767,  35.59881 ,
         34.891106]], dtype=float32)
Coordinates:
  * x        (x) float64 1.0 2.0 3.0 4.0 5.0 6.0 ... 44.0 45.0 46.0 47.0 48.0
  * y        (y) float64 1.0 2.0 3.0 4.0 5.0 6.0 ... 44.0 45.0 46.0 47.0 48.0
Attributes:
    cell_methods:  time: point
    long_name:     latitude
    units:         degrees_N

In [3]:
grid = catalog["grid/c24"].read()
grid.lat[1]

KeyError: 'grid/c24'

In [1]:
# load graph data
from dgl.contrib.data import load_data
import torch

data = load_data(dataset='aifb')
num_nodes = data.num_nodes
num_rels = data.num_rels
num_classes = data.num_classes
labels = data.labels
train_idx = data.train_idx
# split training and validation set
val_idx = train_idx[:len(train_idx) // 5]
train_idx = train_idx[len(train_idx) // 5:]

# edge type and normalization factor
edge_type = torch.from_numpy(data.edge_type)
edge_norm = torch.from_numpy(data.edge_norm).unsqueeze(1)

Loading dataset aifb
Number of nodes:  8285
Number of edges:  66371
Number of relations:  91
Number of classes:  4
removing nodes that are more than 3 hops away


In [21]:
import numpy as np
np.max(edge_norm)

TypeError: max() received an invalid combination of arguments - got (out=NoneType, axis=NoneType, ), but expected one of:
 * ()
 * (Tensor other)
 * (int dim, bool keepdim)
      didn't match because some of the keywords were incorrect: out, axis
 * (name dim, bool keepdim)
      didn't match because some of the keywords were incorrect: out, axis


In [2]:
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import dgl.function as fn

g = DGLGraph((data.edge_src, data.edge_dst))


/home/ebrahimn/miniconda3/envs/fv3net/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


In [11]:
g.edges()

(tensor([   0, 4230, 5147,  ..., 4215, 5493, 8284]),
 tensor([   0,    0,    0,  ..., 8284, 8284, 8284]))

In [6]:
g

Graph(num_nodes=8285, num_edges=65439,
      ndata_schemes={}
      edata_schemes={})

In [8]:

g.edata.update({'rel_type': edge_type, 'norm': edge_norm})

{}

In [17]:
len(edge_type)

65439

In [13]:
edge_type[0:40]

tensor([ 0, 55, 55,  0, 39,  0, 30, 18, 19,  8,  1,  6, 18, 19,  1,  6,  1,  6,
        18, 19,  4, 24, 14, 15,  4,  1,  6,  8, 12,  8,  0, 10,  2,  5,  4,  0,
        13,  0,  4,  1])

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn import SAGEConv
from dgl.nn.pytorch import GraphConv, ChebConv, GATConv
import numpy as np
import dask.diagnostics
import fsspec
import xarray as xr
import matplotlib.pyplot as plt
import networkx as nx
import os
from torch.utils.data import DataLoader
import torch.optim as optim
import warnings
import time
import select as sl
import pickle
from load_data import *
from utils import *
from Newmodel import *
import wandb
from fv3net.artifacts.resolve_url import resolve_url
from vcm import get_fs

lead=1

lead2=1
day=1
control_str='TATNTATN'#'TNSTTNST' #'TNTSTNTST'
epochs=10
num_heads=2 

variableList=['h500','h200','h850']
TotalSamples=8500
Chuncksize=500



lr=0.001
disablecuda ='store_true'
batch_size=1
window=24*day
pred_len=1
channels=[7, 16, 32, 64, 32, 128]
drop_prob = 0
out_feat=2

savemodelpath = (
    "weight_layer_"
    + control_str
    + "_lag"
    + str(lead)
    +"_lead"
    +str(lead2)
    + "_window"
    + str(window)
    + "_epochs_"
    + str(epochs)
    + ".pt"
)

BUCKET = "vcm-ml-experiments"
PROJECT = "full-model-emulation"

model_out_url = resolve_url(BUCKET, PROJECT, savemodelpath)
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/"


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/" 
state_training_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "atmos_dt_atmos.zarr"))
)
# state_training_data2 = xr.open_zarr(fsspec.get_mapper(os.path.join(data_url, 'sfc_dt_atmos.zarr')))
lat_lon_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "state_after_timestep.zarr"))
)

landSea = xr.open_zarr(
    fsspec.get_mapper(
        "gs://vcm-ml-experiments/default/2022-05-09/baseline-35day-spec-sst/fv3gfs_run/state_after_timestep.zarr"
    )
)
landSea_Mask = landSea.land_sea_mask[1].load()
landSea_Mask = landSea_Mask[:, ::, ::].values.flatten()


lat=(lat_lon_data.latitude[1].load())
lon=(lat_lon_data.longitude[1].load())
lat=lat[:,::,::].values.flatten()
lon=lon[:,::,::].values.flatten()
# cosLat=np.expand_dims(np.cos(lat),axis=1)
# cosLon=np.expand_dims(np.cos(lon),axis=1)
# sinLat=np.expand_dims(np.sin(lat),axis=1)
# sinLon=np.expand_dims(np.sin(lon),axis=1)
cosLat=np.cos(lat)
cosLon=np.cos(lon)
sinLat=np.sin(lat)
sinLon=np.sin(lon)
for i in range(3):
        if i==0:
            sinLon=torch.tensor(sinLon).unsqueeze(0).repeat(int(window/lead),1)
            cosLon=torch.tensor(cosLon).unsqueeze(0).repeat(int(window/lead),1)
            sinLat=torch.tensor(sinLat).unsqueeze(0).repeat(int(window/lead),1)
            cosLat=torch.tensor(cosLat).unsqueeze(0).repeat(int(window/lead),1)
            landSea_Mask=torch.tensor(landSea_Mask).unsqueeze(0).repeat(int(window/lead),1)
        elif i==1:
            sinLon=(sinLon).unsqueeze(0).repeat(1,1,1)
            cosLon=(cosLon).unsqueeze(0).repeat(1,1,1)
            sinLat=(sinLat).unsqueeze(0).repeat(1,1,1)
            cosLat=(cosLat).unsqueeze(0).repeat(1,1,1)
            landSea_Mask=(landSea_Mask).unsqueeze(0).repeat(1,1,1)

        elif i==2:
            sinLon=(sinLon).unsqueeze(0).repeat(batch_size,1,1,1)
            cosLon=(cosLon).unsqueeze(0).repeat(batch_size,1,1,1)
            sinLat=(sinLat).unsqueeze(0).repeat(batch_size,1,1,1)
            cosLat=(cosLat).unsqueeze(0).repeat(batch_size,1,1,1)
            landSea_Mask=(landSea_Mask).unsqueeze(0).repeat(batch_size,1,1,1)

exteraVar=torch.cat((sinLon, sinLat,cosLon,cosLat,landSea_Mask), 1).to(device)

num_nodes=len(lon)
print(f"numebr of grids: {num_nodes}")



g = pickle.load(open("UpdatedGraph", 'rb'))
loss = nn.MSELoss()
g = g.to(device)
model = STGCN_WAVE(channels, int(window/lead), out_feat, num_nodes, g, drop_prob, device, num_heads, control_str).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)
model.train()

Zmean=5765.8457   #Z500mean=5765.8457, 
Zstd=90.79599   #Z500std=90.79599

Tmean=10643.382          #Thickmean=10643.382
Tstd=162.12427              #Thickstd=162.12427
valInde=0


all_indices=np.random.permutation(np.arange(start=0, stop=int(TotalSamples/Chuncksize)))

/home/ebrahimn/miniconda3/envs/dlwp2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


numebr of grids: 13824


In [ ]:
for epoch in range(1, epochs + 1):
    
    for ss in all_indices:

        Z500train=state_training_data[variableList[0]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize))
        T2mtrain1=state_training_data[variableList[1]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize))
        T2mtrain2=state_training_data[variableList[2]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize))

        Z500train=np.swapaxes(Z500train.values, 1, 0)
        T2mtrain1=np.swapaxes(T2mtrain1.values, 1, 0)
        T2mtrain2=np.swapaxes(T2mtrain2.values, 1, 0)

        T2mtrain=T2mtrain1-T2mtrain2

        T2mtrain=T2mtrain.reshape(np.size(T2mtrain, 0), np.size(T2mtrain, 1)*np.size(T2mtrain, 2)*np.size(T2mtrain, 3))
        Z500train=Z500train.reshape(np.size(Z500train, 0), np.size(Z500train, 1)*np.size(Z500train, 2)*np.size(Z500train, 3))

        # Zmean = np.mean(Z500train)
        # Zstd = np.std(Z500train)

        # Tmean = np.mean(T2mtrain)
        # Tstd = np.std(T2mtrain)


        T2mtrain=(T2mtrain-Tmean)/Tstd
        Z500train=(Z500train-Zmean)/Zstd

        

        T2mtrain=np.expand_dims(T2mtrain,axis=0)
        Z500train=np.expand_dims(Z500train,axis=0)

        dataSets=np.concatenate((Z500train,T2mtrain),axis=0)

        num_samples=np.size(dataSets,1)
        print(f"Total samples: {num_samples}")


        len_val = round(num_samples * 0.25)
        len_train = round(num_samples * 0.75)
        train = dataSets[:,: len_train]
        val = dataSets[:,len_train+14: len_train + len_val]

        x_train, y_train = data_transform(train, window, pred_len, lead, lead2, device)
        x_val, y_val = data_transform(val, window, pred_len, lead, lead2, device)
        # x_test, y_test = data_transform(test, n_his, n_pred, device)
        print('size of training:',np.shape(x_train),' size of validation',np.shape(x_val))
        train_data = torch.utils.data.TensorDataset(x_train, y_train)
        train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
        val_data = torch.utils.data.TensorDataset(x_val, y_val)
        val_iter = torch.utils.data.DataLoader(val_data, batch_size)
        # test_data = torch.utils.data.TensorDataset(x_test, y_test)
        # test_iter = torch.utils.data.DataLoader(test_data, batch_size)


        

        if valInde==0:
            min_val_loss = np.inf
            valInde+=1

        l_sum, n = 0.0, 0
        for x, y in train_iter:
            exteraVar1=exteraVar[:x.size(0)]
            x=torch.cat((x, exteraVar1), 1).float()
            y_pred = model(x).view(len(x), out_feat, -1)
            l = loss(y_pred, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
            print("section ",ss," epoch", epoch, ", train loss:", l.item())

        scheduler.step()
        val_loss = evaluate_model(model, loss, val_iter,exteraVar,out_feat)
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            torch.save(model.state_dict(), savemodelpath)
        print("epoch", epoch, ", train loss:", l_sum / n, ", validation loss:", val_loss)

        fs = get_fs(model_out_url)
        fs.put(savemodelpath, model_out_url)
        print(savemodelpath, model_out_url)



# best_model = STGCN_WAVE(channels, window, num_nodes, g, drop_prob, num_layers, device, control_str).to(device)
# best_model.load_state_dict(torch.load(savemodelpath))

# l = evaluate_model(best_model, loss, test_iter)
# MAE, MAPE, RMSE = evaluate_metric(best_model, test_iter, scaler)
# print("test loss:", l, "\nMAE:", MAE, ", MAPE:", MAPE, ", RMSE:", RMSE)


In [ ]:
for epoch in range(1, epochs + 1):
        ss=0
        Z500train=state_training_data[variableList[0]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize))
        T2mtrain1=state_training_data[variableList[1]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize))
        T2mtrain2=state_training_data[variableList[2]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize))

        Z500train=np.swapaxes(Z500train.values, 1, 0)
        T2mtrain1=np.swapaxes(T2mtrain1.values, 1, 0)
        T2mtrain2=np.swapaxes(T2mtrain2.values, 1, 0)

        T2mtrain=T2mtrain1-T2mtrain2

        T2mtrain=T2mtrain.reshape(np.size(T2mtrain, 0), np.size(T2mtrain, 1)*np.size(T2mtrain, 2)*np.size(T2mtrain, 3))
        Z500train=Z500train.reshape(np.size(Z500train, 0), np.size(Z500train, 1)*np.size(Z500train, 2)*np.size(Z500train, 3))


        T2mtrain=(T2mtrain-Tmean)/Tstd
        Z500train=(Z500train-Zmean)/Zstd

        

        T2mtrain=np.expand_dims(T2mtrain,axis=0)
        Z500train=np.expand_dims(Z500train,axis=0)
        dataSets=np.concatenate((Z500train,T2mtrain),axis=0)

        num_samples=np.size(dataSets,1)
        print(f"Total samples: {num_samples}")


        len_train = round(num_samples * 0.75)
        train = dataSets[:,: len_train]

        x_train, y_train = data_transform(train, window, pred_len, lead, lead2, device)
        print('size of training:',np.shape(x_train),' size of validation',np.shape(x_val))
        train_data = torch.utils.data.TensorDataset(x_train, y_train)
        train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)1

        for x, y in train_iter:
            exteraVar1=exteraVar[:x.size(0)]
            x=torch.cat((x, exteraVar1), 1).float()
            x.requires_grad_()
            y_pred = model(x).view(len(x), out_feat, -1)
            y_pred.sum().backward()

            # print the gradient of random index  
            print(x.grad.reshape(-1)[np.random.randint(0,x.grad.nelement())])

            #make sure that none of the indices are zero
            if np.any(x.grad.cpu().detach().numpy()==0):
                print('Global dependency issue')
                break


In [ ]:
# import torch library
import torch

# create tensors with requires_grad = true
x=torch.rand(3, 3)
x.requires_grad_()
A=torch.eye(3)


y = A*x

y.sum().backward()
# Access the gradients using x.grad
dx = x.grad
print("x.grad :", dx)

if np.any(x.grad.cpu().detach().numpy()==0):
    print('Global dependency issue')

In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn import SAGEConv
from dgl.nn.pytorch import GraphConv, ChebConv, GATConv
import numpy as np
import dask.diagnostics
import fsspec
import xarray as xr
import matplotlib.pyplot as plt
import networkx as nx
import os
from torch.utils.data import DataLoader
import torch.optim as optim
import warnings
import time
import select as sl
import pickle
from load_data import *
from utils import *
from Newmodel import *
import wandb
from fv3net.artifacts.resolve_url import resolve_url
from vcm import get_fs

lead=1
lead2=4
day=1
coarsenInd=3

control_str='MPGNN'#'TNSTTNST' #'TNTSTNTST'

print(control_str)

epochs=20
num_heads=2 

variableList=['h500','h200','h850']
TotalSamples=8500
Chuncksize=500

lr=0.001
disablecuda ='store_true'
batch_size=2
window=1
pred_len=1
out_feat=2

savemodelpath = (
    "weight_layer_"
    + control_str
    + "_lag"
    + str(lead)
    +"_lead"
    +str(lead2)
    + "_window"
    + str(window)
    + "_epochs_"
    + str(epochs)
    + ".pt"
)

BUCKET = "vcm-ml-experiments"
PROJECT = "full-model-emulation"

model_out_url = resolve_url(BUCKET, PROJECT, savemodelpath)
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/"


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/" 
state_training_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "atmos_dt_atmos.zarr")), consolidated=True
)
# state_training_data2 = xr.open_zarr(fsspec.get_mapper(os.path.join(data_url, 'sfc_dt_atmos.zarr')))
lat_lon_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "state_after_timestep.zarr"))
)

landSea = xr.open_zarr(
    fsspec.get_mapper(
        "gs://vcm-ml-experiments/default/2022-05-09/baseline-35day-spec-sst/fv3gfs_run/state_after_timestep.zarr"
    )
)
landSea_Mask = landSea.land_sea_mask[1].load()
landSea_Mask = landSea_Mask[:, ::coarsenInd, ::coarsenInd].values.flatten()


lat=(lat_lon_data.latitude[1].load())
lon=(lat_lon_data.longitude[1].load())
lat=lat[:,::coarsenInd,::coarsenInd].values.flatten()
lon=lon[:,::coarsenInd,::coarsenInd].values.flatten()
# cosLat=np.expand_dims(np.cos(lat),axis=1)
# cosLon=np.expand_dims(np.cos(lon),axis=1)
# sinLat=np.expand_dims(np.sin(lat),axis=1)
# sinLon=np.expand_dims(np.sin(lon),axis=1)
cosLat=np.cos(lat)
cosLon=np.cos(lon)
sinLat=np.sin(lat)
sinLon=np.sin(lon)
for i in range(3):
        if i==0:
            sinLon=torch.tensor(sinLon).unsqueeze(0).repeat(int(window/lead),1)
            cosLon=torch.tensor(cosLon).unsqueeze(0).repeat(int(window/lead),1)
            sinLat=torch.tensor(sinLat).unsqueeze(0).repeat(int(window/lead),1)
            cosLat=torch.tensor(cosLat).unsqueeze(0).repeat(int(window/lead),1)
            landSea_Mask=torch.tensor(landSea_Mask).unsqueeze(0).repeat(int(window/lead),1)
        elif i==1:
            sinLon=(sinLon).unsqueeze(0).repeat(1,1,1)
            cosLon=(cosLon).unsqueeze(0).repeat(1,1,1)
            sinLat=(sinLat).unsqueeze(0).repeat(1,1,1)
            cosLat=(cosLat).unsqueeze(0).repeat(1,1,1)
            landSea_Mask=(landSea_Mask).unsqueeze(0).repeat(1,1,1)

        elif i==2:
            sinLon=(sinLon).unsqueeze(0).repeat(batch_size,1,1,1)
            cosLon=(cosLon).unsqueeze(0).repeat(batch_size,1,1,1)
            sinLat=(sinLat).unsqueeze(0).repeat(batch_size,1,1,1)
            cosLat=(cosLat).unsqueeze(0).repeat(batch_size,1,1,1)
            landSea_Mask=(landSea_Mask).unsqueeze(0).repeat(batch_size,1,1,1)

exteraVar=torch.cat((sinLon, sinLat,cosLon,cosLat,landSea_Mask), 1).to(device)

num_nodes=len(lon)
print(f"numebr of grids: {num_nodes}")



g = pickle.load(open("UpdatedGraph_Neighbour10_Coarsen3", 'rb'))
loss = nn.MSELoss()
g = g.to(device)

Zmean=5765.8457   #Z500mean=5765.8457, 
Zstd=90.79599   #Z500std=90.79599

Tmean=10643.382          #Thickmean=10643.382
Tstd=162.12427              #Thickstd=162.12427
valInde=0


class MPNNGNN(nn.Module):
    """MPNN.

    MPNN is introduced in `Neural Message Passing for Quantum Chemistry
    <https://arxiv.org/abs/1704.01212>`__.

    This class performs message passing in MPNN and returns the updated node representations.

    Parameters
    ----------
    node_in_feats : int
        Size for the input node features.
    node_out_feats : int
        Size for the output node representations. Default to 64.
    edge_in_feats : int
        Size for the input edge features. Default to 128.
    edge_hidden_feats : int
        Size for the hidden edge representations.
    num_step_message_passing : int
        Number of message passing steps. Default to 6.
    """
    def __init__(self, node_in_feats, edge_in_feats, node_hidden_feats=64,node_out_feats=64,
                 edge_hidden_feats=128, num_step_message_passing=6):
        super(MPNNGNN, self).__init__()

        self.project_node_feats = nn.Sequential(
            nn.Linear(node_in_feats, node_hidden_feats),
            nn.ReLU(),
            nn.Linear(node_hidden_feats, node_hidden_feats)
        )
        self.num_step_message_passing = num_step_message_passing
        edge_network = nn.Sequential(
            nn.Linear(edge_in_feats, edge_hidden_feats),
            nn.ReLU(),
            nn.Linear(edge_hidden_feats, node_hidden_feats * node_hidden_feats)
        )
        self.gnn_layer = NNConv(
            in_feats=node_hidden_feats,
            out_feats=node_hidden_feats,
            edge_func=edge_network,
            aggregator_type='sum'
        )
        self.gru = nn.GRU(node_hidden_feats, node_hidden_feats)

        self.decoder = nn.Sequential(nn.Linear( node_hidden_feats , node_hidden_feats),
                              nn.ReLU(),
                              nn.Linear( node_hidden_feats, node_out_feats)
                              )

    def reset_parameters(self):
        """Reinitialize model parameters."""
        self.project_node_feats[0].reset_parameters()
        self.gnn_layer.reset_parameters()
        for layer in self.gnn_layer.edge_func:
            if isinstance(layer, nn.Linear):
                layer.reset_parameters()
        self.gru.reset_parameters()


    def forward(self, g, node_feats, edge_feats):
        """Performs message passing and updates node representations.

        Parameters
        ----------
        g : DGLGraph
            DGLGraph for a batch of graphs.
        node_feats : float32 tensor of shape (V, node_in_feats)
            Input node features. V for the number of nodes in the batch of graphs.
        edge_feats : float32 tensor of shape (E, edge_in_feats)
            Input edge features. E for the number of edges in the batch of graphs.

        Returns
        -------
        node_feats : float32 tensor of shape (V, node_out_feats)
            Output node representations.
        """
        node_feats = self.project_node_feats(node_feats) # (V, node_out_feats)
        hidden_feats = node_feats.unsqueeze(0)           # (1, V, node_out_feats)

        for _ in range(self.num_step_message_passing):
            # print(node_feats.shape)
            node_feats = F.relu(self.gnn_layer(g, node_feats, edge_feats))
            node_feats, hidden_feats = self.gru(node_feats.unsqueeze(0), hidden_feats)
            node_feats = node_feats.squeeze(0)

        return self.decoder(node_feats)
        


all_indices=np.random.permutation(np.arange(start=0, stop=int(TotalSamples/Chuncksize)))


for epoch in range(1, 2):
    
        ss=0

        Z500train=state_training_data[variableList[0]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain1=state_training_data[variableList[1]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain2=state_training_data[variableList[2]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()

        Z500train=np.swapaxes(Z500train.values, 1, 0)
        T2mtrain1=np.swapaxes(T2mtrain1.values, 1, 0)
        T2mtrain2=np.swapaxes(T2mtrain2.values, 1, 0)

        T2mtrain=T2mtrain1-T2mtrain2

        T2mtrain=T2mtrain.reshape(np.size(T2mtrain, 0), np.size(T2mtrain, 1)*np.size(T2mtrain, 2)*np.size(T2mtrain, 3))
        Z500train=Z500train.reshape(np.size(Z500train, 0), np.size(Z500train, 1)*np.size(Z500train, 2)*np.size(Z500train, 3))

        # Zmean = np.mean(Z500train)
        # Zstd = np.std(Z500train)

        # Tmean = np.mean(T2mtrain)
        # Tstd = np.std(T2mtrain)


        T2mtrain=(T2mtrain-Tmean)/Tstd
        Z500train=(Z500train-Zmean)/Zstd

        

        T2mtrain=np.expand_dims(T2mtrain,axis=0)
        Z500train=np.expand_dims(Z500train,axis=0)

        dataSets=np.concatenate((Z500train,T2mtrain),axis=0)

        num_samples=np.size(dataSets,1)
        print(f"Total samples: {num_samples}")


        len_val = round(num_samples * 0.25)
        len_train = round(num_samples * 0.75)
        train = dataSets[:,: len_train]
        val = dataSets[:,len_train+14: len_train + len_val]

        x_train=train[:,0:-lead,:]
        y_train=train[:,lead::,:]
        x_val=val[:,0:-lead,:]
        y_val=val[:,lead::,:]


MPGNN
numebr of grids: 1536
Total samples: 500


In [10]:
x_train=train[:,0:-lead,:]
y_train=train[:,lead::,:]
x_val=val[:,0:-lead,:]
y_val=val[:,lead::,:]


x_train=torch.Tensor(x_train).to(device)
y_train=torch.Tensor(y_train).to(device)
x_train=np.swapaxes(x_train, 1, 0)
y_train=np.swapaxes(y_train, 1, 0)
train_data = torch.utils.data.TensorDataset(x_train, y_train)
train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)


x_val=torch.Tensor(x_val).to(device)
y_val=torch.Tensor(y_val).to(device)
x_val=np.swapaxes(x_val, 1, 0)
y_val=np.swapaxes(y_val, 1, 0)
val_data = torch.utils.data.TensorDataset(x_val, y_val)
val_iter = torch.utils.data.DataLoader(val_data, batch_size)


In [18]:
np.shape(y_train)

(2, 374, 1536)

In [14]:
np.shape(val)

(2, 111, 1536)

In [37]:
x_train=torch.Tensor(x_train).to(device)
y_train=torch.Tensor(y_train).to(device)
x_train=np.swapaxes(x_train, 1, 0)
y_train=np.swapaxes(y_train, 1, 0)
train_data = torch.utils.data.TensorDataset(x_train, y_train)
train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)


x_val=torch.Tensor(x_val).to(device)
y_val=torch.Tensor(y_val).to(device)
x_val=np.swapaxes(x_val, 1, 0)
y_val=np.swapaxes(y_val, 1, 0)
val_data = torch.utils.data.TensorDataset(x_val, y_val)
val_iter = torch.utils.data.DataLoader(val_data, batch_size)


In [45]:
for x, y in train_iter:
        print(x.shape)

torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([10, 2, 1536])
torch.Size([4, 2, 1536])


In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn import SAGEConv
from dgl.nn.pytorch import NNConv
import numpy as np
import dask.diagnostics
import fsspec
import xarray as xr
import matplotlib.pyplot as plt
import networkx as nx
import os
from torch.utils.data import DataLoader
import torch.optim as optim
import warnings
import time
import select as sl
import pickle
from load_data import *
from utils import *
from Newmodel import *
import wandb
from fv3net.artifacts.resolve_url import resolve_url
from vcm import get_fs

lead=4
coarsenInd=3
control_str='MPGNN'#'TNSTTNST' #'TNTSTNTST'

print(control_str)

epochs=20
num_heads=2 

variableList=['h500','h200','h850']
TotalSamples=8500
Chuncksize=500



lr=0.001
disablecuda ='store_true'
batch_size=1
drop_prob = 0
out_feat=2

savemodelpath = (
    "weight_layer_"
    + control_str
    + "_lead"
    + str(lead)
    + "_epochs_"
    + str(epochs)
    + ".pt"
)

BUCKET = "vcm-ml-experiments"
PROJECT = "full-model-emulation"

model_out_url = resolve_url(BUCKET, PROJECT, savemodelpath)
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/"


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/" 
state_training_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "atmos_dt_atmos.zarr")), consolidated=True
)
# state_training_data2 = xr.open_zarr(fsspec.get_mapper(os.path.join(data_url, 'sfc_dt_atmos.zarr')))
lat_lon_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "state_after_timestep.zarr"))
)

landSea = xr.open_zarr(
    fsspec.get_mapper(
        "gs://vcm-ml-experiments/default/2022-05-09/baseline-35day-spec-sst/fv3gfs_run/state_after_timestep.zarr"
    )
)
landSea_Mask = landSea.land_sea_mask[1].load()
landSea_Mask = landSea_Mask[:, ::coarsenInd, ::coarsenInd].values.flatten()


lat=(lat_lon_data.latitude[1].load())
lon=(lat_lon_data.longitude[1].load())
lat=lat[:,::coarsenInd,::coarsenInd].values.flatten()
lon=lon[:,::coarsenInd,::coarsenInd].values.flatten()
# cosLat=np.expand_dims(np.cos(lat),axis=1)
# cosLon=np.expand_dims(np.cos(lon),axis=1)
# sinLat=np.expand_dims(np.sin(lat),axis=1)
# sinLon=np.expand_dims(np.sin(lon),axis=1)
cosLat=np.cos(lat)
cosLon=np.cos(lon)
sinLat=np.sin(lat)
sinLon=np.sin(lon)
for i in range(3):
        if i==0:
            sinLon=torch.tensor(sinLon).unsqueeze(0).repeat(1,1)
            cosLon=torch.tensor(cosLon).unsqueeze(0).repeat(1,1)
            sinLat=torch.tensor(sinLat).unsqueeze(0).repeat(1,1)
            cosLat=torch.tensor(cosLat).unsqueeze(0).repeat(1,1)
            landSea_Mask=torch.tensor(landSea_Mask).unsqueeze(0).repeat(1,1)
        elif i==2:
            sinLon=(sinLon).unsqueeze(0).repeat(batch_size,1,1)
            cosLon=(cosLon).unsqueeze(0).repeat(batch_size,1,1)
            sinLat=(sinLat).unsqueeze(0).repeat(batch_size,1,1)
            cosLat=(cosLat).unsqueeze(0).repeat(batch_size,1,1)
            landSea_Mask=(landSea_Mask).unsqueeze(0).repeat(batch_size,1,1)

exteraVar=torch.cat((sinLon, sinLat,cosLon,cosLat,landSea_Mask), 1).to(device)
exteraVar=np.swapaxes(exteraVar,2, 1)

num_nodes=len(lon)
print(f"numebr of grids: {num_nodes}")



g = pickle.load(open("UpdatedGraph_Neighbour10_Coarsen3", 'rb'))

edg=np.asarray(g.edges())
latInd=lat[edg[1]]
lonInd=lon[edg[1]]
latlon=[latInd.T,lonInd.T]
# latlon=np.swapaxes(latlon, 1, 0)
latlon=torch.from_numpy(np.swapaxes(latlon, 1, 0)).float()



Zmean=5765.8457   #Z500mean=5765.8457, 
Zstd=90.79599   #Z500std=90.79599

Tmean=10643.382          #Thickmean=10643.382
Tstd=162.12427              #Thickstd=162.12427
valInde=0




# best_model = STGCN_WAVE(channels, window, num_nodes, g, drop_prob, num_layers, device, control_str).to(device)
# best_model.load_state_dict(torch.load(savemodelpath))

# l = evaluate_model(best_model, loss, test_iter)
# MAE, MAPE, RMSE = evaluate_metric(best_model, test_iter, scaler)
# print("test loss:", l, "\nMAE:", MAE, ", MAPE:", MAPE, ", RMSE:", RMSE)


/home/ebrahimn/miniconda3/envs/dlwp2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MPGNN
numebr of grids: 1536


/tmp/ipykernel_1534820/3689726334.py:121: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  edg=np.asarray(g.edges())
/tmp/ipykernel_1534820/3689726334.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edg=np.asarray(g.edges())


In [2]:

class MPNNGNN(nn.Module):
    """MPNN.

    MPNN is introduced in `Neural Message Passing for Quantum Chemistry
    <https://arxiv.org/abs/1704.01212>`__.

    This class performs message passing in MPNN and returns the updated node representations.

    Parameters
    ----------
    node_in_feats : int
        Size for the input node features.
    node_out_feats : int
        Size for the output node representations. Default to 64.
    edge_in_feats : int
        Size for the input edge features. Default to 128.
    edge_hidden_feats : int
        Size for the hidden edge representations.
    num_step_message_passing : int
        Number of message passing steps. Default to 6.
    """
    def __init__(self, g, node_in_feats, edge_in_feats, node_hidden_feats=64,node_out_feats=64,
                 edge_hidden_feats=128, num_step_message_passing=6):
        super(MPNNGNN, self).__init__()

        self.project_node_feats = nn.Sequential(
            nn.Linear(node_in_feats, node_hidden_feats),
            nn.ReLU(),
            nn.Linear(node_hidden_feats, node_hidden_feats),
            nn.LayerNorm(node_hidden_feats)
        )
        self.num_step_message_passing = num_step_message_passing
        edge_network = nn.Sequential(
            nn.Linear(edge_in_feats, edge_hidden_feats),
            nn.ReLU(),
            nn.Linear(edge_hidden_feats, node_hidden_feats * node_hidden_feats),
            nn.LayerNorm(node_hidden_feats * node_hidden_feats)
        )
        self.gnn_layer = NNConv(
            in_feats=node_hidden_feats,
            out_feats=node_hidden_feats,
            edge_func=edge_network,
            aggregator_type='sum'
        )
        self.gru = nn.GRU(node_hidden_feats, node_hidden_feats)

        self.decoder = nn.Sequential(nn.Linear( node_hidden_feats , node_hidden_feats),
                              nn.ReLU(),
                              nn.Linear( node_hidden_feats, node_out_feats)
                              )
        self.g=g

    def reset_parameters(self):
        """Reinitialize model parameters."""
        self.project_node_feats[0].reset_parameters()
        self.gnn_layer.reset_parameters()
        for layer in self.gnn_layer.edge_func:
            if isinstance(layer, nn.Linear):
                layer.reset_parameters()
        self.gru.reset_parameters()


    def forward(self, node_feats, edge_feats):
        """Performs message passing and updates node representations.

        Parameters
        ----------
        g : DGLGraph
            DGLGraph for a batch of graphs.
        node_feats : float32 tensor of shape (V, node_in_feats)
            Input node features. V for the number of nodes in the batch of graphs.
        edge_feats : float32 tensor of shape (E, edge_in_feats)
            Input edge features. E for the number of edges in the batch of graphs.

        Returns
        -------
        node_feats : float32 tensor of shape (V, node_out_feats)
            Output node representations.
        """
        node_feats = self.project_node_feats(node_feats) # (V, node_out_feats)
        hidden_feats = node_feats.unsqueeze(0)           # (1, V, node_out_feats)

        for _ in range(self.num_step_message_passing):
            print(node_feats.shape)
            print(hidden_feats.shape)

            node_feats = F.relu(self.gnn_layer(self.g, node_feats, edge_feats))
            node_feats, hidden_feats = self.gru(node_feats.unsqueeze(0), hidden_feats)
            node_feats = node_feats.squeeze(0)

        return self.decoder(node_feats)
        


loss = nn.MSELoss()
g = g.to(device)
model = MPNNGNN(g,node_in_feats=7, edge_in_feats=2, edge_hidden_feats=256, node_hidden_feats=256, node_out_feats=2,num_step_message_passing=8).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)
model.train()

all_indices=np.random.permutation(np.arange(start=0, stop=int(TotalSamples/Chuncksize)))

In [5]:
for epoch in range(1, epochs + 1):
    
    for ss in all_indices:

        Z500train=state_training_data[variableList[0]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain1=state_training_data[variableList[1]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain2=state_training_data[variableList[2]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()

        Z500train=np.swapaxes(Z500train.values, 1, 0)
        T2mtrain1=np.swapaxes(T2mtrain1.values, 1, 0)
        T2mtrain2=np.swapaxes(T2mtrain2.values, 1, 0)

        T2mtrain=T2mtrain1-T2mtrain2

        T2mtrain=T2mtrain.reshape(np.size(T2mtrain, 0), np.size(T2mtrain, 1)*np.size(T2mtrain, 2)*np.size(T2mtrain, 3))
        Z500train=Z500train.reshape(np.size(Z500train, 0), np.size(Z500train, 1)*np.size(Z500train, 2)*np.size(Z500train, 3))

        # Zmean = np.mean(Z500train)
        # Zstd = np.std(Z500train)

        # Tmean = np.mean(T2mtrain)
        # Tstd = np.std(T2mtrain)


        T2mtrain=(T2mtrain-Tmean)/Tstd
        Z500train=(Z500train-Zmean)/Zstd

        

        T2mtrain=np.expand_dims(T2mtrain,axis=0)
        Z500train=np.expand_dims(Z500train,axis=0)

        dataSets=np.concatenate((Z500train,T2mtrain),axis=0)

        num_samples=np.size(dataSets,1)
        print(f"Total samples: {num_samples}")


        len_val = round(num_samples * 0.25)
        len_train = round(num_samples * 0.75)
        train = dataSets[:,: len_train]
        val = dataSets[:,len_train+14: len_train + len_val]

        x_train=train[:,0:-lead,:]
        y_train=train[:,lead::,:]
        x_val=val[:,0:-lead,:]
        y_val=val[:,lead::,:]


        x_train=torch.Tensor(x_train).to(device)
        y_train=torch.Tensor(y_train).to(device)
        x_train=np.swapaxes(x_train, 1, 0)
        y_train=np.swapaxes(y_train, 1, 0)
        x_train=np.swapaxes(x_train, 2, 1)
        y_train=np.swapaxes(y_train, 2, 1)

        train_data = torch.utils.data.TensorDataset(x_train, y_train)
        train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)


        x_val=torch.Tensor(x_val).to(device)
        y_val=torch.Tensor(y_val).to(device)
        x_val=np.swapaxes(x_val, 1, 0)
        y_val=np.swapaxes(y_val, 1, 0)
        x_val=np.swapaxes(x_val, 2, 1)
        y_val=np.swapaxes(y_val, 2, 1)

        val_data = torch.utils.data.TensorDataset(x_val, y_val)
        val_iter = torch.utils.data.DataLoader(val_data, batch_size)
        

        if valInde==0:
            min_val_loss = np.inf
            valInde+=1

        l_sum, n = 0.0, 0
        cc=0
        for x, y in train_iter:
            cc+=1
        print(cc)

Total samples: 500
371
Total samples: 500
371
Total samples: 500
371


KeyboardInterrupt: 

In [ ]:
for epoch in range(1, epochs + 1):
    
    for ss in all_indices:

        Z500train=state_training_data[variableList[0]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain1=state_training_data[variableList[1]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain2=state_training_data[variableList[2]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()

        Z500train=np.swapaxes(Z500train.values, 1, 0)
        T2mtrain1=np.swapaxes(T2mtrain1.values, 1, 0)
        T2mtrain2=np.swapaxes(T2mtrain2.values, 1, 0)

        T2mtrain=T2mtrain1-T2mtrain2

        T2mtrain=T2mtrain.reshape(np.size(T2mtrain, 0), np.size(T2mtrain, 1)*np.size(T2mtrain, 2)*np.size(T2mtrain, 3))
        Z500train=Z500train.reshape(np.size(Z500train, 0), np.size(Z500train, 1)*np.size(Z500train, 2)*np.size(Z500train, 3))

        # Zmean = np.mean(Z500train)
        # Zstd = np.std(Z500train)

        # Tmean = np.mean(T2mtrain)
        # Tstd = np.std(T2mtrain)


        T2mtrain=(T2mtrain-Tmean)/Tstd
        Z500train=(Z500train-Zmean)/Zstd

        

        T2mtrain=np.expand_dims(T2mtrain,axis=0)
        Z500train=np.expand_dims(Z500train,axis=0)

        dataSets=np.concatenate((Z500train,T2mtrain),axis=0)

        num_samples=np.size(dataSets,1)
        print(f"Total samples: {num_samples}")


        len_val = round(num_samples * 0.25)
        len_train = round(num_samples * 0.75)
        train = dataSets[:,: len_train]
        val = dataSets[:,len_train+14: len_train + len_val]

        x_train=train[:,0:-lead,:]
        y_train=train[:,lead::,:]
        x_val=val[:,0:-lead,:]
        y_val=val[:,lead::,:]


        x_train=torch.Tensor(x_train).to(device)
        y_train=torch.Tensor(y_train).to(device)
        x_train=np.swapaxes(x_train, 1, 0)
        y_train=np.swapaxes(y_train, 1, 0)
        x_train=np.swapaxes(x_train, 2, 1)
        y_train=np.swapaxes(y_train, 2, 1)

        train_data = torch.utils.data.TensorDataset(x_train, y_train)
        train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)


        x_val=torch.Tensor(x_val).to(device)
        y_val=torch.Tensor(y_val).to(device)
        x_val=np.swapaxes(x_val, 1, 0)
        y_val=np.swapaxes(y_val, 1, 0)
        x_val=np.swapaxes(x_val, 2, 1)
        y_val=np.swapaxes(y_val, 2, 1)

        val_data = torch.utils.data.TensorDataset(x_val, y_val)
        val_iter = torch.utils.data.DataLoader(val_data, batch_size)
        

        if valInde==0:
            min_val_loss = np.inf
            valInde+=1

        l_sum, n = 0.0, 0
        for x, y in train_iter:
            exteraVar1=exteraVar[:x.size(0)]
            x=torch.squeeze(torch.cat((x, exteraVar1), 2)).float()
            y_pred = model(x,latlon).view(len(x), -1 ,out_feat)
            l = loss(y_pred, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
            print("section ",ss," epoch", epoch, ", train loss:", l.item())

        scheduler.step()
        val_loss = evaluate_model(model, loss, val_iter,exteraVar,out_feat)
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            torch.save(model.state_dict(), savemodelpath)
        print("epoch", epoch, ", train loss:", l_sum / n, ", validation loss:", val_loss)

        fs = get_fs(model_out_url)
        fs.put(savemodelpath, model_out_url)
        print(savemodelpath, model_out_url)

In [16]:
x=squeeze(x)

NameError: name 'squeeze' is not defined

In [14]:
x.shape

AttributeError: 'builtin_function_or_method' object has no attribute 'shape'

In [4]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn import SAGEConv
from dgl.nn.pytorch import NNConv
import numpy as np
import dask.diagnostics
import fsspec
import xarray as xr
import matplotlib.pyplot as plt
import networkx as nx
import os
from torch.utils.data import DataLoader
import torch.optim as optim
import warnings
import time
import select as sl
import pickle
from load_data import *
from utilsMPGNNUnet import *
import wandb
from fv3net.artifacts.resolve_url import resolve_url
from vcm import get_fs

lead=6

coarsenInd=1
g = pickle.load(open("UpdatedGraph_Neighbour10", 'rb'))
residual=0
num_step=15
aggregat='mean'

control_str='MultiSAGEUnet'#'TNSTTNST' #'TNTSTNTST'

print(control_str)

epochs=50

variableList=['h500','h200','h850']
TotalSamples=8500
Chuncksize=20



lr=0.001
disablecuda ='store_true'
batch_size=1
drop_prob = 0
out_feat=2

savemodelpath = (
    "weight_layer_"
    + control_str
    + "_lead"
    + str(lead)
    + "_epochs_"
    + str(epochs)
    +"MP_Block_"
    +str(num_step)
    + "aggregat_"
    +aggregat
    +"coarsen_"
    +str(coarsenInd)
    +"residual_"
    +str(residual)
    +".pt"
)

print(savemodelpath)

BUCKET = "vcm-ml-experiments"
PROJECT = "full-model-emulation"

model_out_url = resolve_url(BUCKET, PROJECT, savemodelpath)
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/"


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/" 
state_training_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "atmos_dt_atmos.zarr")), consolidated=True
)
# state_training_data2 = xr.open_zarr(fsspec.get_mapper(os.path.join(data_url, 'sfc_dt_atmos.zarr')))
lat_lon_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "state_after_timestep.zarr"))
)

landSea = xr.open_zarr(
    fsspec.get_mapper(
        "gs://vcm-ml-experiments/default/2022-05-09/baseline-35day-spec-sst/fv3gfs_run/state_after_timestep.zarr"
    )
)
landSea_Mask = landSea.land_sea_mask[1].load()
landSea_Mask = landSea_Mask[:, ::coarsenInd, ::coarsenInd].values.flatten()


lat=(lat_lon_data.latitude[1].load())
lon=(lat_lon_data.longitude[1].load())
lat=lat[:,::coarsenInd,::coarsenInd].values.flatten()
lon=lon[:,::coarsenInd,::coarsenInd].values.flatten()
# cosLat=np.expand_dims(np.cos(lat),axis=1)
# cosLon=np.expand_dims(np.cos(lon),axis=1)
# sinLat=np.expand_dims(np.sin(lat),axis=1)
# sinLon=np.expand_dims(np.sin(lon),axis=1)
cosLat=np.cos(lat)
cosLon=np.cos(lon)
sinLat=np.sin(lat)
sinLon=np.sin(lon)
for i in range(2):
        if i==0:
            sinLon=torch.tensor(sinLon).unsqueeze(0).repeat(1,1)
            cosLon=torch.tensor(cosLon).unsqueeze(0).repeat(1,1)
            sinLat=torch.tensor(sinLat).unsqueeze(0).repeat(1,1)
            cosLat=torch.tensor(cosLat).unsqueeze(0).repeat(1,1)
            landSea_Mask=torch.tensor(landSea_Mask).unsqueeze(0).repeat(1,1)
        elif i==1:
            sinLon=(sinLon).unsqueeze(0).repeat(batch_size,1,1)
            cosLon=(cosLon).unsqueeze(0).repeat(batch_size,1,1)
            sinLat=(sinLat).unsqueeze(0).repeat(batch_size,1,1)
            cosLat=(cosLat).unsqueeze(0).repeat(batch_size,1,1)
            landSea_Mask=(landSea_Mask).unsqueeze(0).repeat(batch_size,1,1)

exteraVar=torch.cat((sinLon, sinLat,cosLon,cosLat,landSea_Mask), 1).to(device)
exteraVar=np.swapaxes(exteraVar,2, 1)
print(device)

num_nodes=len(lon)
print(f"numebr of grids: {num_nodes}")




edg=np.asarray(g.edges())
latInd=lat[edg[1]]
lonInd=lon[edg[1]]
latlon=[latInd.T,lonInd.T]
# latlon=np.swapaxes(latlon, 1, 0)
latlon=torch.from_numpy(np.swapaxes(latlon, 1, 0)).float()
latlon=latlon.to(device)


Zmean=5765.8457   #Z500mean=5765.8457, 
Zstd=90.79599   #Z500std=90.79599

Tmean=10643.382          #Thickmean=10643.382
Tstd=162.12427              #Thickstd=162.12427
valInde=0

print('loading model')

class UnetGraphSAGE(nn.Module):
    def __init__(self, g, in_feats, h_feats,out_feat,num_step,aggregat):
        super(UnetGraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats,aggregat)
        self.conv2 = SAGEConv(h_feats, int(h_feats/2), aggregat)
        self.conv3 = SAGEConv(int(h_feats/2), int(h_feats/4), aggregat)
        self.conv4 = SAGEConv(int(h_feats/4), int(h_feats/4), aggregat)
        self.conv5 = SAGEConv(int(h_feats/2), int(h_feats/2), aggregat)
        self.conv6 = SAGEConv(h_feats, out_feat,aggregat)
        self.g=g
        self.num_step=num_step
        
    def forward(self, in_feat,exteraVar1):

        for _ in range(self.num_step):
            h = self.conv1(self.g, in_feat)
            h = F.relu(h)
            h = self.conv2(self.g, h)
            h = F.relu(h)
            h = self.conv3(self.g, h)
            h = F.relu(h)
            tuple = (self.conv4(self.g, h),h)
            h = torch.cat(tuple,dim=1)
            h = F.relu(h)
            tuple = (self.conv5(self.g, h),h)
            h = torch.cat(tuple,dim=1)
            h = F.relu(h)
            h = self.conv6(self.g, h)
            in_feat=torch.cat((h, torch.squeeze(exteraVar1)), 1).float()
        return h
        

loss = nn.MSELoss()
g = g.to(device)
model = UnetGraphSAGE(g,7,256, 2,num_step,aggregat).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)
model.train()

all_indices=np.random.permutation(np.arange(start=0, stop=int(TotalSamples/Chuncksize)))


for epoch in range(1, epochs + 1):
    
    for ss in all_indices:

        Z500train=state_training_data[variableList[0]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain1=state_training_data[variableList[1]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain2=state_training_data[variableList[2]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()

        Z500train=np.swapaxes(Z500train.values, 1, 0)
        T2mtrain1=np.swapaxes(T2mtrain1.values, 1, 0)
        T2mtrain2=np.swapaxes(T2mtrain2.values, 1, 0)

        T2mtrain=T2mtrain1-T2mtrain2

        T2mtrain=T2mtrain.reshape(np.size(T2mtrain, 0), np.size(T2mtrain, 1)*np.size(T2mtrain, 2)*np.size(T2mtrain, 3))
        Z500train=Z500train.reshape(np.size(Z500train, 0), np.size(Z500train, 1)*np.size(Z500train, 2)*np.size(Z500train, 3))

        # Zmean = np.mean(Z500train)
        # Zstd = np.std(Z500train)

        # Tmean = np.mean(T2mtrain)
        # Tstd = np.std(T2mtrain)


        T2mtrain=(T2mtrain-Tmean)/Tstd
        Z500train=(Z500train-Zmean)/Zstd

        

        T2mtrain=np.expand_dims(T2mtrain,axis=0)
        Z500train=np.expand_dims(Z500train,axis=0)

        dataSets=np.concatenate((Z500train,T2mtrain),axis=0)

        num_samples=np.size(dataSets,1)
        print(f"Total samples: {num_samples}")


        len_val = round(num_samples * 0.25)
        len_train = round(num_samples * 0.75)
        train = dataSets[:,: len_train]
        val = dataSets[:,len_train+14: len_train + len_val]

        x_train=train[:,0:-2*lead,:]
        y_train=train[:,lead:-lead,:]
        y_train2=train[:,lead::,:]

        x_val=val[:,0:-lead,:]
        y_val=val[:,lead::,:]

        if residual==1:
            y_train=y_train-x_train
            y_val=y_val-x_val


        x_train=np.swapaxes(x_train, 1, 0)
        y_train=np.swapaxes(y_train, 1, 0)
        y_train2=np.swapaxes(y_train2, 1, 0)

        x_train=np.swapaxes(x_train, 2, 1)
        y_train=np.swapaxes(y_train, 2, 1)
        y_train2=np.swapaxes(y_train2, 2, 1)

        x_train=torch.Tensor(x_train).to(device)
        y_train=torch.Tensor(y_train).to(device)
        y_train2=torch.Tensor(y_train2).to(device)

        train_data = torch.utils.data.TensorDataset(x_train, y_train, y_train2)
        train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)



        x_val=np.swapaxes(x_val, 1, 0)
        y_val=np.swapaxes(y_val, 1, 0)
        x_val=np.swapaxes(x_val, 2, 1)
        y_val=np.swapaxes(y_val, 2, 1)
        x_val=torch.Tensor(x_val).to(device)
        y_val=torch.Tensor(y_val).to(device)


        val_data = torch.utils.data.TensorDataset(x_val, y_val)
        val_iter = torch.utils.data.DataLoader(val_data, batch_size)
        

        if valInde==0:
            min_val_loss = np.inf
            valInde+=1

        l_sum, n = 0.0, 0
        for x, y, y2 in train_iter:
            print(x-y)

MultiSAGEUnet
weight_layer_MultiSAGEUnet_lead6_epochs_50MP_Block_15aggregat_meancoarsen_1residual_0.pt
cpu
numebr of grids: 13824
loading model


/tmp/ipykernel_1713075/3128626329.py:135: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  edg=np.asarray(g.edges())
/tmp/ipykernel_1713075/3128626329.py:135: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edg=np.asarray(g.edges())


Total samples: 20
tensor([[[-0.1136,  0.0388],
         [-0.0921,  0.0382],
         [-0.0869,  0.0354],
         ...,
         [ 0.0797,  0.0729],
         [ 0.0854,  0.0642],
         [ 0.0697,  0.0870]]])
tensor([[[ 0.0280,  0.0669],
         [ 0.0654,  0.0742],
         [ 0.0707,  0.0749],
         ...,
         [ 0.0078,  0.0728],
         [ 0.0075,  0.0579],
         [-0.0144,  0.0727]]])
tensor([[[-0.0476,  0.0615],
         [-0.0156,  0.0670],
         [-0.0059,  0.0653],
         ...,
         [ 0.0520,  0.0833],
         [ 0.0509,  0.0651],
         [ 0.0335,  0.0809]]])
Total samples: 20
tensor([[[-0.0943, -0.0038],
         [-0.1135, -0.0195],
         [-0.1123, -0.0122],
         ...,
         [-0.0109,  0.0399],
         [-0.0155,  0.0038],
         [-0.0407, -0.0448]]])
tensor([[[-0.1116,  0.0241],
         [-0.1303,  0.0131],
         [-0.1294,  0.0196],
         ...,
         [-0.0305, -0.0294],
         [ 0.0013, -0.0069],
         [-0.0247, -0.0700]]])
tensor([[[-0.0

KeyboardInterrupt: 

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn import SAGEConv
from dgl.nn.pytorch import NNConv
import numpy as np
import dask.diagnostics
import fsspec
import xarray as xr
import matplotlib.pyplot as plt
import networkx as nx
import os
from torch.utils.data import DataLoader
import torch.optim as optim
import warnings
import time
import select as sl
import pickle
from load_data import *
from utilsMPGNNUnet import *
import wandb
from fv3net.artifacts.resolve_url import resolve_url
from vcm import get_fs

lead=6
multiStep=5

coarsenInd=1
g = pickle.load(open("UpdatedGraph_Neighbour10", 'rb'))
residual=0
num_step=4
aggregat='mean'

control_str='MultiSAGEUnet'#'TNSTTNST' #'TNTSTNTST'

print(control_str)

epochs=50

variableList=['h500','h200','h850']
TotalSamples=8500
Chuncksize=500



lr=0.001
disablecuda ='store_true'
batch_size=1
drop_prob = 0
out_feat=2

savemodelpath = (
    "weight_layer_"
    + control_str
    + "_lead"
    + str(lead)
    + "_epochs_"
    + str(epochs)
    +"MP_Block_"
    +str(num_step)
    + "aggregat_"
    +aggregat
    +"coarsen_"
    +str(coarsenInd)
    +"residual_"
    +str(residual)
    +".pt"
)

print(savemodelpath)

BUCKET = "vcm-ml-experiments"
PROJECT = "full-model-emulation"

model_out_url = resolve_url(BUCKET, PROJECT, savemodelpath)
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/"


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/" 
state_training_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "atmos_dt_atmos.zarr")), consolidated=True
)
# state_training_data2 = xr.open_zarr(fsspec.get_mapper(os.path.join(data_url, 'sfc_dt_atmos.zarr')))
lat_lon_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "state_after_timestep.zarr"))
)

landSea = xr.open_zarr(
    fsspec.get_mapper(
        "gs://vcm-ml-experiments/default/2022-05-09/baseline-35day-spec-sst/fv3gfs_run/state_after_timestep.zarr"
    )
)
landSea_Mask = landSea.land_sea_mask[1].load()
landSea_Mask = landSea_Mask[:, ::coarsenInd, ::coarsenInd].values.flatten()


lat=(lat_lon_data.latitude[1].load())
lon=(lat_lon_data.longitude[1].load())
lat=lat[:,::coarsenInd,::coarsenInd].values.flatten()
lon=lon[:,::coarsenInd,::coarsenInd].values.flatten()
# cosLat=np.expand_dims(np.cos(lat),axis=1)
# cosLon=np.expand_dims(np.cos(lon),axis=1)
# sinLat=np.expand_dims(np.sin(lat),axis=1)
# sinLon=np.expand_dims(np.sin(lon),axis=1)
cosLat=np.cos(lat)
cosLon=np.cos(lon)
sinLat=np.sin(lat)
sinLon=np.sin(lon)
for i in range(2):
        if i==0:
            sinLon=torch.tensor(sinLon).unsqueeze(0).repeat(1,1)
            cosLon=torch.tensor(cosLon).unsqueeze(0).repeat(1,1)
            sinLat=torch.tensor(sinLat).unsqueeze(0).repeat(1,1)
            cosLat=torch.tensor(cosLat).unsqueeze(0).repeat(1,1)
            landSea_Mask=torch.tensor(landSea_Mask).unsqueeze(0).repeat(1,1)
        elif i==1:
            sinLon=(sinLon).unsqueeze(0).repeat(batch_size,1,1)
            cosLon=(cosLon).unsqueeze(0).repeat(batch_size,1,1)
            sinLat=(sinLat).unsqueeze(0).repeat(batch_size,1,1)
            cosLat=(cosLat).unsqueeze(0).repeat(batch_size,1,1)
            landSea_Mask=(landSea_Mask).unsqueeze(0).repeat(batch_size,1,1)

exteraVar=torch.cat((sinLon, sinLat,cosLon,cosLat,landSea_Mask), 1).to(device)
exteraVar=np.swapaxes(exteraVar,2, 1)
print(device)

num_nodes=len(lon)
print(f"numebr of grids: {num_nodes}")




edg=np.asarray(g.edges())
latInd=lat[edg[1]]
lonInd=lon[edg[1]]
latlon=[latInd.T,lonInd.T]
# latlon=np.swapaxes(latlon, 1, 0)
latlon=torch.from_numpy(np.swapaxes(latlon, 1, 0)).float()
latlon=latlon.to(device)


Zmean=5765.8457   #Z500mean=5765.8457, 
Zstd=90.79599   #Z500std=90.79599

Tmean=10643.382          #Thickmean=10643.382
Tstd=162.12427              #Thickstd=162.12427
valInde=0

print('loading model')

class UnetGraphSAGE(nn.Module):
    def __init__(self, g, in_feats, h_feats,out_feat,num_step,aggregat):
        super(UnetGraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats,aggregat)
        self.conv2 = SAGEConv(h_feats, int(h_feats/2), aggregat)
        self.conv3 = SAGEConv(int(h_feats/2), int(h_feats/4), aggregat)
        self.conv4 = SAGEConv(int(h_feats/4), int(h_feats/4), aggregat)
        self.conv5 = SAGEConv(int(h_feats/2), int(h_feats/2), aggregat)
        self.conv6 = SAGEConv(h_feats, out_feat,aggregat)
        self.g=g
        self.num_step=num_step
        
    def forward(self, in_feat,exteraVar1):

        for _ in range(self.num_step):
            h = self.conv1(self.g, in_feat)
            h = F.relu(h)
            h = self.conv2(self.g, h)
            h = F.relu(h)
            h = self.conv3(self.g, h)
            h = F.relu(h)
            tuple = (self.conv4(self.g, h),h)
            h = torch.cat(tuple,dim=1)
            h = F.relu(h)
            tuple = (self.conv5(self.g, h),h)
            h = torch.cat(tuple,dim=1)
            h = F.relu(h)
            h = self.conv6(self.g, h)
            in_feat=torch.cat((h, torch.squeeze(exteraVar1)), 1).float()
        return h
        

loss = nn.MSELoss()
g = g.to(device)
model = UnetGraphSAGE(g,7,256, 2,num_step,aggregat).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)
model.train()

all_indices=np.random.permutation(np.arange(start=0, stop=int(TotalSamples/Chuncksize)))



/home/ebrahimn/miniconda3/envs/dlwp2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MultiSAGEUnet
weight_layer_MultiSAGEUnet_lead6_epochs_50MP_Block_4aggregat_meancoarsen_1residual_0.pt
cpu
numebr of grids: 13824
loading model


/tmp/ipykernel_1890812/1716582478.py:136: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  edg=np.asarray(g.edges())
/tmp/ipykernel_1890812/1716582478.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edg=np.asarray(g.edges())


In [2]:

for epoch in range(1,2):
    
        ss=0

        Z500train=state_training_data[variableList[0]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain1=state_training_data[variableList[1]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain2=state_training_data[variableList[2]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()

        Z500train=np.swapaxes(Z500train.values, 1, 0)
        T2mtrain1=np.swapaxes(T2mtrain1.values, 1, 0)
        T2mtrain2=np.swapaxes(T2mtrain2.values, 1, 0)

        T2mtrain=T2mtrain1-T2mtrain2

        T2mtrain=T2mtrain.reshape(np.size(T2mtrain, 0), np.size(T2mtrain, 1)*np.size(T2mtrain, 2)*np.size(T2mtrain, 3))
        Z500train=Z500train.reshape(np.size(Z500train, 0), np.size(Z500train, 1)*np.size(Z500train, 2)*np.size(Z500train, 3))

        # Zmean = np.mean(Z500train)
        # Zstd = np.std(Z500train)

        # Tmean = np.mean(T2mtrain)
        # Tstd = np.std(T2mtrain)


        T2mtrain=(T2mtrain-Tmean)/Tstd
        Z500train=(Z500train-Zmean)/Zstd

        

        T2mtrain=np.expand_dims(T2mtrain,axis=0)
        Z500train=np.expand_dims(Z500train,axis=0)

        dataSets=np.concatenate((Z500train,T2mtrain),axis=0)

        num_samples=np.size(dataSets,1)
        print(f"Total samples: {num_samples}")


        len_val = round(num_samples * 0.25)
        len_train = round(num_samples * 0.75)
        train = dataSets[:,: len_train]
        val = dataSets[:,len_train+14: len_train + len_val]

        x_train=train[:,0:-multiStep*lead,:]

        y_train=x_t=np.zeros([np.size(train,0),np.size(train,1)-multiStep*lead,np.size(train,2),multiStep])
        for sm in range(1,multiStep+1):
            y_train[:,:,:,sm]=train[:,(sm+1)*lead:-(multiStep-1)*lead,:]
        print(y_train[:,:,:,-1])
        x_val=val[:,0:-lead,:]
        y_val=val[:,lead::,:]



        x_train=np.swapaxes(x_train, 1, 0)
        y_train=np.swapaxes(y_train, 1, 0)

        x_train=np.swapaxes(x_train, 2, 1)
        y_train=np.swapaxes(y_train, 2, 1)

        x_train=torch.Tensor(x_train)
        y_train=torch.Tensor(y_train)
        y_train2=torch.Tensor(y_train2)

        
        train_data = torch.utils.data.TensorDataset(x_train, y_train)
        train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)

/home/ebrahimn/miniconda3/envs/dlwp2/lib/python3.8/site-packages/numpy/core/_methods.py:179: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


Total samples: 500


ValueError: could not broadcast input array from shape (2,339,13824) into shape (2,345,13824)

In [24]:
for sm in range(1,multiStep+1):
    print(multiStep-sm)
    if sm==multiStep:
        y_train[:,:,:,sm-1]=train[:,(sm)*lead::,:]
    else:
        y_train[:,:,:,sm-1]=train[:,(sm)*lead:-(multiStep-sm)*lead,:]


4
3
2
1
0


In [27]:
x_train=np.swapaxes(x_train, 1, 0)
y_train=np.swapaxes(y_train, 1, 0)

x_train=np.swapaxes(x_train, 2, 1)
y_train=np.swapaxes(y_train, 2, 1)

x_train=torch.Tensor(x_train)
y_train=torch.Tensor(y_train)


In [31]:
np.shape(x_train)

torch.Size([345, 13824, 2])

In [28]:
train_data = torch.utils.data.TensorDataset(x_train, y_train)


In [29]:
train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)

In [35]:
for x,y in train_iter:
    print(f"x:{np.shape(x)}")
    print(f"y:{np.shape(y)}")

x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5])
x:torch.Size([1, 13824, 2])
y:torch.Size([1, 13824, 2, 5

In [34]:
x[:,:,0]

tensor([[-0.0677, -0.2603, -0.2732,  ...,  0.4969,  0.4378,  0.3226]])